**Script to obtain the data files that are used to initialize the simulations of transcriptional and translational divergence of yeast duplicated genes**

In [31]:
import numpy as np
import pandas as pd
from google.colab import files
import io
import scipy.stats as stats
import math

In [32]:
# Mounting my google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Transcription and translation rates from Hausser et al. (2019) are first imported.

In [33]:
rates_yeast = pd.read_csv('/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_init/yeastRates.csv', sep=';', decimal=',')
rates_yeast = rates_yeast.rename(columns={'Unnamed: 0': 'Gene'})

In [34]:
rates_yeast

,Gene,lm,lp,wRPF,wmRNA,bmEser,amEser,bm,m,bp,cv,apExp,am,isEssential,hasTATA,YEPDFit,pEst
0,TFC3,3.553033,3.064832,1.258158,1.577492,1.125383,0.704432,1.127728,0.423025,2.201503,NaN,-1.057717,0.704302,True,True,0.040325,2.496036
1,VPS8,3.582631,3.105510,0.758155,1.345766,1.146577,0.868402,0.896002,0.191299,1.892547,NaN,NaN,0.704302,False,False,0.020725,1.955355
2,EFB1,2.994317,2.315970,3.849116,3.664817,2.307282,0.058252,3.215053,2.510350,3.453998,NaN,NaN,0.704302,True,False,0.420089,5.835856
3,ERP2,2.811575,2.334454,2.321122,2.327195,1.701383,0.057926,1.877432,1.172729,3.245141,-0.744004,-0.071219,0.704302,False,False,-0.026295,4.289378
4,FUN14,2.775974,2.298853,1.980231,1.698622,0.790021,0.382707,1.248859,0.544156,3.568424,NaN,0.204003,0.704302,False,False,-0.018375,3.984088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4435,OPT2,3.420616,2.943495,-0.236572,0.842609,-0.399593,0.274546,0.392846,-0.311857,1.562993,NaN,NaN,0.704302,False,True,-0.009625,1.122644
4436,SGE1,3.212720,2.735599,1.472025,1.521922,1.158760,0.532465,1.072159,0.367456,2.800172,NaN,NaN,0.704302,False,False,-0.009200,3.039136
4437,ARR1,2.946943,2.469822,1.138934,1.592954,1.449894,0.730236,1.143190,0.438487,2.661827,NaN,NaN,0.704302,False,False,-0.016960,2.971822
4438,ARR2,2.594393,2.117271,0.387390,1.169086,0.400387,0.863932,0.719323,0.014620,2.686701,NaN,NaN,0.704302,False,True,-0.010200,2.572829


## 1) Addition of systematic names

The file saccharomyces_cerevisiae.20200611.gff has been downloaded from Saccharomyces Genome Database (SGD). It contains all genomic features in the S288C reference genome, as of 2020-06-11, as well as the complete genome sequence. It is used to obtain all the systematic - standard name associations.

In [35]:
features_SGD = pd.read_table('/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_init/saccharomyces_cerevisiae.20200611.gff', 
                             skiprows=20, header=None, index_col=False, 
                             names=['seqname', 'source', 'feature', 'start', 
                                    'end', 'score', 'strand', 'frame', 
                                    'attribute'], nrows=23067)
# The genome sequence is not read

features_genes = features_SGD[features_SGD['feature'] == 'gene']
features_genes = features_genes.reset_index(drop=True)

In [36]:
# The features_genes file is used to build a dictionary of standard name -
# systematic name associations

genes_orf = {}

for row in range(features_genes.shape[0]):

  features = features_genes.at[row, 'attribute'].split(';')
  feat_dict = {}

  for feat in features:
    feat_split = feat.split('=')

    feat_dict[feat_split[0]] = feat_split[1]

  if 'gene' in feat_dict.keys(): # For cases when the two IDs are the same 
                                 # (ex: mitochondrial genes)

    genes_orf[feat_dict['gene']] = feat_dict['ID']

  else:

    genes_orf[feat_dict['ID']] = feat_dict['ID']

In [37]:
# This dictionary is used to add the systematic ID of genes to the rates df
rates_yeast.insert(0, 'ORF', 'problem')
missing = []

for row in range(rates_yeast.shape[0]):
  gene = rates_yeast.at[row, 'Gene']

  if gene in genes_orf.keys():
    rates_yeast.at[row, 'ORF'] = genes_orf[gene]
  
  else:

    missing.append(gene)

print(missing)

['AIM1', 'PHO88', 'DUR1,2', 'ARG5,6', 'FRA2', 'ADE5,7', 'AML1', 'YSP1', 'PET10', 'ENV10', 'MDN1', 'FLD1', 'OSW5', 'ICY2']


In [38]:
# For the missing genes, corresponding to alternative aliases, the systematic 
# IDs are added manually

missing_ID = {'AIM1': 'YAL046C', 'PHO88': 'YBR106W', 'DUR1,2': 'YBR208C', 
              'ARG5,6': 'YER069W', 'FRA2': 'YGL220W', 'ADE5,7': 'YGL234W',
              'AML1': 'YGR001C', 'YSP1': 'YHR155W', 'PET10': 'YKR046C',
              'ENV10': 'YLR065C', 'MDN1': 'YLR106C', 'FLD1': 'YLR404W',
              'OSW5': 'YMR148W', 'ICY2': 'YPL250C'}

for gene in missing:

  row = rates_yeast[rates_yeast['Gene'] == gene].index

  rates_yeast.at[row, 'ORF'] = missing_ID[gene]

## 2) Addition of duplication-type annotations

Table S1 from Supplementary file 2 of Marchant et al. (2019) is imported. In this file, each yeast ORF (systematic ID) is associated to a duplication type annotation (S: singleton, SSD: small scale duplication,  SSD-successive: successive small-scale duplication, WGD: whole-genome duplication or SSD_WGD: paralogs duplicated by both mechanism of duplication SSD and WGD).

In [39]:
dupli_type = pd.read_csv('/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_init/TableS1_Marchant_2019.csv', sep=';')

In [40]:
rates_yeast['Duplication'] = 'Unspecified' # For cases that are not in
                                           # the dupli_type dataframe

for row in range(rates_yeast.shape[0]):
  orf = rates_yeast.at[row, 'ORF']

  if orf in list(dupli_type['ORF']):
    dupli = dupli_type[dupli_type['ORF'] == orf]['Duplication'].values[0]
    rates_yeast.at[row, 'Duplication'] = dupli

## 3) Recalculation of transcription and translation rates and rates_Hausser.csv export

We recalculated all expression rates reported by Hausser et al. (2019) from the raw data deposited by Weinberg et al. This was done because we mistakenly thought that the calculation of translation rates by Hausser et al. was erroneous. 

The recalculated translation rates are not used in our work, but some of the preliminary simulations were performed using both the original and recalculated ("corrected") sets of translations rates. In these, the choice of the dataset had no impact on the results.

**Note:** The recalculated transcription rates have been used in the simulations, simply because the script was at first defined that way. They are equivalent to the rates originally reported by Hausser et al. (2019).

Transcription and translation rates, as well as protein abundance, are recalculated from the RPKMs reported by Weinberg et al. (2016), following the Methods of Hausser et al. (2019).

Raw data are downloaded from GSE75897 (https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE75897). The RNA-seq RPKM from the RiboZero experiment (GSE75897_RiboZero_RPKMs.txt.gz) and the ribosome profiling RPKMs (GSE75897_RPF_RPKMs.txt.gz) are obtained and imported:

In [41]:
mRNA_RPKM = pd.read_csv('/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_init/GSE75897_RiboZero_RPKMs.txt', sep='\t', names=['ORF', 'mRNA RPKM'])
RP_RPKM = pd.read_csv('/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_init/GSE75897_RPF_RPKMs.txt', sep='\t', names=['ORF', 'RP RPKM'])

In [42]:
# To obtain a file with only genes for which RPKMs have been reported in
# both experiments

RPKM_all = pd.merge(mRNA_RPKM, RP_RPKM, how='outer')

RPKM_both = RPKM_all.dropna().reset_index(drop=True)

# RPKM of 0.0 are also dropped
RPKM_both = RPKM_both[(RPKM_both['mRNA RPKM'] != 0) & (RPKM_both['RP RPKM'] != 0)].reset_index(drop=True)

There are 5068 genes for which both types of RPKMs are available, compared with 4400 in the dataset from Hausser et al. (2019). This seems consistent with the filtering step described in the Methods of this paper.

In [43]:
# Constants from Hausser et al. (2019)
alpha_m = 5.1
alpha_p = 1.34
n_mRNA = 60000
n_prot = 1.1e8

# Transcription and translation rates are calculated (Hausser et al., 2019)
RPKM_both['m_est'] = np.log10(n_mRNA*(RPKM_both['mRNA RPKM']/np.sum(RPKM_both['mRNA RPKM'])))
RPKM_both['bm_calc'] = np.log10((10**RPKM_both['m_est']) * alpha_m)

RPKM_both['bp_calc'] = np.log10(((n_prot * alpha_p)/(10**RPKM_both['m_est']))*((RPKM_both['RP RPKM'])/np.sum(RP_RPKM['RP RPKM'])))

# As well as a new estimated protein abundance
RPKM_both['pEst_calc'] = np.log10(((10**RPKM_both['bm_calc']) * (10**RPKM_both['bp_calc'])) / (alpha_m * alpha_p))

In [44]:
# Merging with the yeastRates dataframe 
rates_corrected = pd.merge(rates_yeast, RPKM_both[['ORF', 'm_est', 'bm_calc', 'bp_calc', 'pEst_calc']], how='left', on=['ORF'])

**Most important:** Recalculation of the transcription rates from Hausser et al. to account for variation in mRNA decay rates, as measured by Eser et al. (2016):

In [45]:
# The decay rates from Eser et al. first need to be corrected for dilution due
# to cell division
rates_corrected['amEser'] = np.log10(10**rates_corrected['amEser'] + (math.log(2)/(99/60)))

In [46]:
rates_corrected['bm_Eser'] = np.log10(10**rates_corrected['m'] * 10**rates_corrected['amEser'])

In [47]:
rates_corrected

,ORF,Gene,lm,lp,wRPF,wmRNA,bmEser,amEser,bm,m,...,isEssential,hasTATA,YEPDFit,pEst,Duplication,m_est,bm_calc,bp_calc,pEst_calc,bm_Eser
0,YAL001C,TFC3,3.553033,3.064832,1.258158,1.577492,1.125383,0.739048,1.127728,0.423025,...,True,True,0.040325,2.496036,SSD,0.423053,1.130623,2.956097,3.252044,1.162073
1,YAL002W,VPS8,3.582631,3.105510,0.758155,1.345766,1.146577,0.892427,0.896002,0.191299,...,False,False,0.020725,1.955355,SSD-successive,0.191327,0.898897,2.687819,2.752041,1.083726
2,YAL003W,EFB1,2.994317,2.315970,3.849116,3.664817,2.307282,0.194135,3.215053,2.510350,...,True,False,0.420089,5.835856,S,2.510378,3.217948,3.459730,5.843003,2.704485
3,YAL007C,ERP2,2.811575,2.334454,2.321122,2.327195,1.701383,0.193896,1.877432,1.172729,...,False,False,-0.026295,4.289378,SSD_WGD,1.172756,1.880326,3.269357,4.315008,1.366624
4,YAL008W,FUN14,2.775974,2.298853,1.980231,1.698622,0.790021,0.452387,1.248859,0.544156,...,False,False,-0.018375,3.984088,S,0.544183,1.251753,3.557038,3.974117,0.996543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4435,YPR194C,OPT2,3.420616,2.943495,-0.236572,0.842609,-0.399593,0.362062,0.392846,-0.311857,...,False,True,-0.009625,1.122644,SSD,-0.311830,0.395740,2.196249,1.757314,0.050205
4436,YPR198W,SGE1,3.212720,2.735599,1.472025,1.521922,1.158760,0.582951,1.072159,0.367456,...,False,False,-0.009200,3.039136,SSD-successive,0.367483,1.075053,3.225533,3.465911,0.950407
4437,YPR199C,ARR1,2.946943,2.469822,1.138934,1.592954,1.449894,0.762928,1.143190,0.438487,...,False,False,-0.016960,2.971822,SSD-successive,0.438514,1.146085,2.821411,3.132820,1.201415
4438,YPR200C,ARR2,2.594393,2.117271,0.387390,1.169086,0.400387,0.888198,0.719323,0.014620,...,False,True,-0.010200,2.572829,SSD,0.014647,0.722217,2.493734,2.381276,0.902818


This is the final rates_Hausser dataframe which was used to initialize the simulations. It is exported:

In [48]:
rates_corrected.to_csv(path_or_buf='/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_sim_ready/rates_Hausser.csv', index=False, header=True)

## 4) Reconstitution of paralog couples and calculation of relative divergence

The P1-P2 associations are taken from Table S9 of Supplementary File 2 from Marchant et al. (2019), which compiles both SSD- and WGD-derived paralog pairs (as well as other types of paralogs such as multi-SSD which are ignored in the current work). 


In [49]:
dupli_pairs = pd.read_csv('/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_init/TableS9_Marchant_2019.csv', sep=';')
dupli_pairs = dupli_pairs.dropna(subset=['Duplication', 'P1', 'P2']).reset_index(drop=True)

In [50]:
# A new dataframe is defined
rates_couples = pd.DataFrame(columns=['Duplication', 'ORF_P1', 'ORF_P2'])

# After merging, the dataframe will have all those columns:
# Duplication: Duplication type from Marchant et al.
# ORF_P1: Systematic name paralog 1
# Gene_P1: Standard name paralog 1
# ORF_P2: Systematic name paralog 2
# Gene_P2: Standard name paralog 2
# Couple: P1-P1 (standard names)
# bm_P1: Log10 transcription rate of P1, in mRNAs/h (Hausser et al.)
# bm_P2: Log10 transcription rate of P2, in mRNAs/h (Hausser et al.)
# m_P1: Log10 mRNA abundance for P1, in mRNAs (Hausser et al.)
# m_P2: Log10 mRNA abundance for P2, in mRNAs (Hausser et al.)
# amEser_P1: Log10 decay rate of mRNA P1, in h^-1 (Eser et al.)
# amEser_P2: Log10 decay rate of mRNA P2, in h^-1 (Eser et al.)
# bm_Eser_P1: Log10 transcription rate of P1 recalculated to account for mRNA
#             decay measurements (Eser et al.), in mRNAs/h (Hausser et al.)
# bm_Eser_P2: Log10 transcription rate of P1 recalculated to account for mRNA
#             decay measurements (Eser et al.), in mRNAs/h (Hausser et al.)
# bp_P1: Log10 Translation rate of P1, in proteins per mRNA per hour (Hausser et al.)
# bp_calc_P1: Recalculated translation rate of P1
# bp_P2: Log10 Translation rate of P2, in proteins per mRNA per hour (Hausser et al.)
# bp_calc_P2: Recalculated translation rate of P2
# pEst_P1: Log10 Estimated protein abundance for P1 (proteins / cell)
# pEst_calc_P1: Log10 Estimated protein abundance for P1, recalculated using the
#               recalculated translation rate (proteins / cell)
# pEst_P2: Log10 Estimated protein abundance for P2 (proteins per cell)
# pEst_calc_P2: Log10 Estimated protein abundance for P2, recalculated using the
#               recalculated translation rate (proteins / cell)
# apBelle_P1: Log10 protein decay rate for P1, in h^-1 (Belle et al.)
# apBelle_P2: Log10 protein decay rate for P2, in h^-1 (Belle et al.)
# cv_P1: Coefficient of variation for P1 (Newman et al., 2006)
# cv_P2: Coefficient of variation for P2 (Newman et al., 2006)

rates_couples['Duplication'] = dupli_pairs['Duplication']
rates_couples['ORF_P1'] = dupli_pairs['P1']
rates_couples['ORF_P2'] = dupli_pairs['P2']

In [51]:
# New versions of the rates_corrected dataframe are prepared to be merged with
# the rates_couples df just created

rates_only = rates_corrected[['ORF', 'Gene', 'bm', 'm', 'amEser', 'bm_Eser',
                              'bp', 'cv', 'pEst', 'apExp', 'Duplication',
                              'bp_calc', 'pEst_calc']].copy()

rates_P1 = rates_only.copy()
rates_P1 = rates_P1.rename(columns={'ORF': 'ORF_P1', 'Gene': 'Gene_P1', 
                                    'bm': 'bm_P1', 'm': 'm_P1', 
                                    'amEser': 'amEser_P1', 
                                    'bm_Eser': 'bm_Eser_P1', 'bp': 'bp_P1', 
                                    'cv': 'cv_P1', 'pEst': 'pEst_P1',
                                    'apExp': 'apBelle_P1', 
                                    'bp_calc': 'bp_calc_P1',
                                    'pEst_calc': 'pEst_calc_P1'})

rates_P2 = rates_only.copy()
rates_P2 = rates_P2.rename(columns={'ORF': 'ORF_P2', 'Gene': 'Gene_P2', 
                                    'bm': 'bm_P2', 'm': 'm_P2',
                                    'amEser': 'amEser_P2',
                                    'bm_Eser': 'bm_Eser_P2', 'bp': 'bp_P2',
                                    'cv': 'cv_P2', 'pEst': 'pEst_P2',
                                    'apExp': 'apBelle_P2',
                                    'bp_calc': 'bp_calc_P2',
                                    'pEst_calc': 'pEst_calc_P2'})

In [52]:
# The dataframe is then filled
rates_couples = pd.merge(rates_couples, rates_P1, how='left',
                         on=['Duplication', 'ORF_P1'])

rates_couples = pd.merge(rates_couples, rates_P2, how='left',
                         on=['Duplication', 'ORF_P2'])

In [53]:
# Keeping only pairs where both paralogs were included in the dataset
rates_couples = rates_couples.dropna(subset=['bm_P1', 'bm_P2', 'bp_P1', 'bp_P2']).reset_index(drop=True)

In [54]:
# Rearranging the columns in the desired order:
rates_couples = rates_couples[['Duplication', 'ORF_P1', 'Gene_P1', 'ORF_P2', 
                               'Gene_P2', 'bm_P1', 'bm_P2', 'm_P1', 'm_P2',
                               'amEser_P1', 'amEser_P2', 'bm_Eser_P1',
                               'bm_Eser_P2', 'bp_P1', 'bp_calc_P1', 'bp_P2',
                               'bp_calc_P2', 'pEst_P1', 'pEst_calc_P1', 
                               'pEst_P2', 'pEst_calc_P2', 'apBelle_P1',
                               'apBelle_P2', 'cv_P1', 'cv_P2']]

In [55]:
# Definition of a function to compute log2 fold-changes of relative divergence

def fold_change(prop_P1, prop_P2, data, name):
  """Function to calculate log2 fold-change for a property between two
     duplicates. The two properties are provided as dataframe columns.
     The last argument is used to specify the name of the new column 
     inserted in the data dataframe."""

  df = data.copy()
  df[name] = np.NaN

  for row in range(df.shape[0]):

    value_P1 = df.at[row, f'{prop_P1}']
    value_P2 = df.at[row, f'{prop_P2}']

    if value_P1 >= value_P2:

      df.at[row, name] = (10**value_P1)/(10**value_P2)

    elif value_P1 < value_P2:

      df.at[row, name] = (10**value_P2)/(10**value_P1)

  df[name] = np.log2(df[name])

  return df

In [56]:
# Calculation of the log2 fold-changes of divergence
rates_logfold = fold_change('bm_P1', 'bm_P2', rates_couples, 'bm_fold_original')
rates_logfold = fold_change('amEser_P1', 'amEser_P2', rates_logfold, 'amEser_fold')
rates_logfold = fold_change('bm_Eser_P1', 'bm_Eser_P2', rates_logfold, 'bm_fold_Eser')
rates_logfold = fold_change('bp_P1', 'bp_P2', rates_logfold, 'bp_fold_original')
rates_logfold = fold_change('pEst_P1', 'pEst_P2', rates_logfold, 'pEst_fold_original')
rates_logfold = fold_change('apBelle_P1', 'apBelle_P2', rates_logfold, 'apBelle_fold')

rates_logfold = fold_change('bp_calc_P1', 'bp_calc_P2', rates_logfold, 'bp_fold_calc')
rates_logfold = fold_change('pEst_calc_P1', 'pEst_calc_P2', rates_logfold, 'pEst_fold_calc')

In [57]:
# Calculation of normalized differences
rates_div_rel = rates_logfold.copy()

rates_div_rel['bm_diff'] = np.abs(10**rates_div_rel['bm_P2'] - 10**rates_div_rel['bm_P1']) / (10**rates_div_rel['bm_P2'] + 10**rates_div_rel['bm_P1'])
rates_div_rel['bm_Eser_diff'] = np.abs(10**rates_div_rel['bm_Eser_P2'] - 10**rates_div_rel['bm_Eser_P1']) / (10**rates_div_rel['bm_Eser_P2'] + 10**rates_div_rel['bm_Eser_P1'])
rates_div_rel['amEser_diff'] = np.abs(10**rates_div_rel['amEser_P2'] - 10**rates_div_rel['amEser_P1']) / (10**rates_div_rel['amEser_P2'] + 10**rates_div_rel['amEser_P1'])
rates_div_rel['bp_diff'] = np.abs(10**rates_div_rel['bp_P2'] - 10**rates_div_rel['bp_P1']) / (10**rates_div_rel['bp_P2'] + 10**rates_div_rel['bp_P1'])
rates_div_rel['pEst_diff'] = np.abs(10**rates_div_rel['pEst_P2'] - 10**rates_div_rel['pEst_P1']) / (10**rates_div_rel['pEst_P2'] + 10**rates_div_rel['pEst_P1'])
rates_div_rel['apBelle_diff'] = np.abs(10**rates_div_rel['apBelle_P2'] - 10**rates_div_rel['apBelle_P1']) / (10**rates_div_rel['apBelle_P2'] + 10**rates_div_rel['apBelle_P1'])
rates_div_rel['bp_calc_diff'] = np.abs(10**rates_div_rel['bp_calc_P2'] - 10**rates_div_rel['bp_calc_P1']) / (10**rates_div_rel['bp_calc_P2'] + 10**rates_div_rel['bp_calc_P1'])
rates_div_rel['pEst_calc_diff'] = np.abs(10**rates_div_rel['pEst_calc_P2'] - 10**rates_div_rel['pEst_calc_P1']) / (10**rates_div_rel['pEst_calc_P2'] + 10**rates_div_rel['pEst_calc_P1'])

This is the final dataframe with log2 fold-changes and normalized differences that is used to generate figure 1 as well as to initialize the simulations (and compare the simulated and empirical data at the end). It is exported as csv:

In [58]:
rates_div_rel.to_csv(path_or_buf='/content/drive/My Drive/Redaction_SA/Figures_2021_06/Data_sim_ready/couples_divergence.csv', index=False, header=True)